# Example: Read 1 Degree Ocean (MOM6) Replay Data, Store to Zar

Example using [MOM6Dataset](generated/ufs2arco.MOM6Dataset.rst) to read data from two cycles of the replay dataset.

In [1]:
import xarray as xr
from datetime import datetime, timedelta

In [2]:
import sys
sys.path.append("..")
from ufs2arco import MOM6Dataset, RegridMOM6

## Setup path to read from

In this case, read replay data from the s3 bucket.
Right now, [MOM6Dataset](generated/ufs2arco.MOM6Dataset.rst) requires a `path_in` callable with 3 entries, so that it can build file paths for a given date (denoting a DA cycle), forecast hours to grab, and file prefixes.

In [3]:
def ocean_path(cycles, forecast_hours, file_prefixes):

    upper = "s3://noaa-ufs-gefsv13replay-pds/1deg"
    cycles = [cycles] if not isinstance(cycles, list) else cycles

    files = []
    for cycle in cycles:
        this_dir = f"{cycle.year:04d}/{cycle.month:02d}/{cycle.year:04d}{cycle.month:02d}{cycle.day:02d}{cycle.hour:02d}"
        for fp in file_prefixes:
            for fhr in forecast_hours:
                this_date = cycle+timedelta(hours=fhr)
                this_file = f"{this_dir}/{fp}{this_date.year:04d}_{this_date.month:02d}_{this_date.day:02d}_{this_date.hour:02d}.nc"
                files.append(this_file)
    return [f"{upper}/{this_file}" for this_file in files]

For instance to grab the `ocn_` files at forecast hour `fhr00` 
from the DA cycle at 00:00 Jan 1, 1994, this would be used as follows:

In [4]:
cycles = [datetime(1994,1,1,0), datetime(1994,1,1,6)]
ocean_path(cycles, [0], ["ocn_"])

['s3://noaa-ufs-gefsv13replay-pds/1deg/1994/01/1994010100/ocn_1994_01_01_00.nc',
 's3://noaa-ufs-gefsv13replay-pds/1deg/1994/01/1994010106/ocn_1994_01_01_06.nc']

However, reading from s3 is super slow with `xarray.open_dataset`.
Luckily we can tell fsspec to cache the files locally before opening, this is denoted by
prepending the names as follows.

See discussion [here](https://discourse.pangeo.io/t/reading-goes-r-s3-netcdfs-from-an-aws-ec2-instance-is-it-possible-to-get-faster-speeds-than-from-my-local-machine/2440/13)
for more info.

In [5]:
def cached_path(cycles, forecast_hours, file_prefixes):
    return [f"simplecache::{u}" for u in ocean_path(cycles, forecast_hours, file_prefixes)]

In [6]:
cached_path(cycles, [0], ["ocn_"])

['simplecache::s3://noaa-ufs-gefsv13replay-pds/1deg/1994/01/1994010100/ocn_1994_01_01_00.nc',
 'simplecache::s3://noaa-ufs-gefsv13replay-pds/1deg/1994/01/1994010106/ocn_1994_01_01_06.nc']

## Use MOM6Dataset

Some of that is wrapped up under the hood in [MOM6Dataset](generated/ufs2arco.MOM6Dataset.rst), we just need to give two inputs:

1. The filename mapping as defined above

2. A configuration yaml file

In [7]:
reader = MOM6Dataset(path_in=cached_path, config_filename="config-replay.yaml")

MOM6Dataset.__init__: Could not find coords_path_out in config-replay.yaml, using default.


We'll use this reader to open a dataset with [.open_dataset()](generated/ufs2arco.MOM6Dataset.open_dataset.rst), note that the `fsspec_kwargs` is not necessary for local file reads, but necessary for reading from buckets like s3

In [8]:
%%time
ds = reader.open_dataset(cycles, fsspec_kwargs={"s3":{"anon":True}}, engine="h5netcdf")
ds

CPU times: user 2.33 s, sys: 1.21 s, total: 3.54 s
Wall time: 50.5 s


/home/daniel/miniconda3/envs/regrid/lib/python3.11/site-packages/xarray/core/dataset.py:282: UserWarning: The specified chunks separate the stored chunks along dimension "z_l" starting at index 1. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/home/daniel/storageu/ufs2arco/docs/../ufs2arco/ufsdataset.py:292: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by converting the values to nanosecond precision ahead of time.
  xtime = xr.DataArray(


<xarray.Dataset>
Dimensions:     (xq: 360, yh: 320, z_l: 75, z_i: 76, time: 2, nv: 2, xh: 360,
                 yq: 320, zl: 75)
Coordinates:
  * xq          (xq) float64 -299.0 -298.0 -297.0 -296.0 ... 57.0 58.0 59.0 60.0
  * yh          (yh) float64 -77.77 -77.56 -77.34 -77.12 ... 89.48 89.69 89.9
  * z_l         (z_l) float64 0.5154 1.571 2.687 ... 5.698e+03 5.902e+03
  * z_i         (z_i) float64 0.0 1.031 2.112 ... 5.596e+03 5.8e+03 6.004e+03
    cftime      (time) object 1994-01-01 00:00:00 1994-01-01 06:00:00
  * nv          (nv) float64 1.0 2.0
  * xh          (xh) float64 -299.5 -298.5 -297.5 -296.5 ... 56.5 57.5 58.5 59.5
  * yq          (yq) float64 -77.67 -77.45 -77.23 -77.01 ... 89.59 89.8 90.0
  * zl          (zl) float64 1.0 3.0 5.0 7.0 ... 3.338e+03 3.591e+03 5.111e+03
  * time        (time) datetime64[ns] 1994-01-01 1994-01-01T06:00:00
    ftime       (time) timedelta64[ns] 00:00:00 00:00:00
Data variables: (12/23)
    uo          (time, z_l, yh, xq) float32 dask.array<chunksize=(1, 1, 320, 360), meta=np.ndarray>
    vo          (time, z_l, yq, xh) float32 dask.array<chunksize=(1, 1, 320, 360), meta=np.ndarray>
    so          (time, z_l, yh, xh) float32 dask.array<chunksize=(1, 1, 320, 360), meta=np.ndarray>
    temp        (time, z_l, yh, xh) float32 dask.array<chunksize=(1, 1, 320, 360), meta=np.ndarray>
    ho          (time, zl, yh, xh) float32 dask.array<chunksize=(1, 9, 320, 360), meta=np.ndarray>
    pbo         (time, yh, xh) float32 dask.array<chunksize=(1, 320, 360), meta=np.ndarray>
    ...          ...
    LwLatSens   (time, yh, xh) float32 dask.array<chunksize=(1, 320, 360), meta=np.ndarray>
    Heat_PmE    (time, yh, xh) float32 dask.array<chunksize=(1, 320, 360), meta=np.ndarray>
    average_T1  (time) datetime64[ns] dask.array<chunksize=(1,), meta=np.ndarray>
    average_T2  (time) datetime64[ns] dask.array<chunksize=(1,), meta=np.ndarray>
    average_DT  (time) timedelta64[ns] dask.array<chunksize=(1,), meta=np.ndarray>
    time_bnds   (time, nv) object dask.array<chunksize=(1, 2), meta=np.ndarray>
Attributes:
    NumFilesInSet:  1
    title:          UFS
    grid_type:      regular
    grid_tile:      N/A

xarray gives a nice view of the data, we can look at a single variable to see how the data are laid out

In [9]:
ds["temp"]

<xarray.DataArray 'temp' (time: 2, z_l: 75, yh: 320, xh: 360)>
dask.array<concatenate, shape=(2, 75, 320, 360), dtype=float32, chunksize=(1, 1, 320, 360), chunktype=numpy.ndarray>
Coordinates:
  * yh       (yh) float64 -77.77 -77.56 -77.34 -77.12 ... 89.26 89.48 89.69 89.9
  * z_l      (z_l) float64 0.5154 1.571 2.687 ... 5.495e+03 5.698e+03 5.902e+03
    cftime   (time) object 1994-01-01 00:00:00 1994-01-01 06:00:00
  * xh       (xh) float64 -299.5 -298.5 -297.5 -296.5 ... 56.5 57.5 58.5 59.5
  * time     (time) datetime64[ns] 1994-01-01 1994-01-01T06:00:00
    ftime    (time) timedelta64[ns] 00:00:00 00:00:00
Attributes:
    units:          degC
    long_name:      Potential Temperature
    cell_methods:   area:mean z_l:mean yh:mean xh:mean time: mean
    time_avg_info:  average_T1,average_T2,average_DT

## Regridding the dataset

We may want to regrid the dataset to a Gaussian grid utilized by FV3.

For this example, we will create a Gaussian grid for this purpose.

It is also possible to read the grid from an FV3 dataset using `RegridMOM6.read_grid()`

In [10]:
lats, lons = RegridMOM6.compute_gaussian_grid(180, 360)
lats

array([-89.23664167, -88.24777411, -87.25306675, -86.25703993,
       -85.26051261, -84.26374309, -83.26683823, -82.26985012,
       -81.2728072 , -80.27572627, -79.27861791, -78.28148911,
       -77.28434468, -76.28718801, -75.2900216 , -74.2928473 ,
       -73.29566652, -72.29848035, -71.30128967, -70.30409514,
       -69.30689734, -68.3096967 , -67.31249362, -66.31528838,
       -65.31808127, -64.32087249, -63.32366223, -62.32645066,
       -61.32923791, -60.3320241 , -59.33480934, -58.33759371,
       -57.3403773 , -56.34316017, -55.34594239, -54.34872402,
       -53.3515051 , -52.35428567, -51.35706578, -50.35984547,
       -49.36262476, -48.36540368, -47.36818227, -46.37096054,
       -45.37373851, -44.37651622, -43.37929366, -42.38207087,
       -41.38484786, -40.38762464, -39.39040122, -38.39317762,
       -37.39595384, -36.39872991, -35.40150582, -34.40428159,
       -33.40705723, -32.40983274, -31.41260814, -30.41538342,
       -29.41815859, -28.42093367, -27.42370865, -26.42

Create the Ocean dataset regridder object from the lats and lons

In [11]:
rg = RegridMOM6(lats, lons, ds, config_filename = "config-replay.yaml")

Regrid the whole dataset

In [12]:
ds = rg.regrid(ds)

In [13]:
ds

<xarray.Dataset>
Dimensions:     (time: 2, lon: 360, lat: 180, lev: 75, lev1: 75, nv: 2)
Coordinates:
    cftime      (time) object 1994-01-01 00:00:00 1994-01-01 06:00:00
    ftime       (time) timedelta64[ns] 00:00:00 00:00:00
  * lon         (lon) float64 0.0 1.0 2.0 3.0 4.0 ... 356.0 357.0 358.0 359.0
  * lat         (lat) float64 -89.24 -88.25 -87.25 -86.26 ... 87.25 88.25 89.24
  * nv          (nv) float64 1.0 2.0
  * lev         (lev) float64 0.5154 1.571 2.687 ... 5.698e+03 5.902e+03
  * lev1        (lev1) float64 1.0 3.0 5.0 7.0 ... 3.338e+03 3.591e+03 5.111e+03
  * time        (time) datetime64[ns] 1994-01-01 1994-01-01T06:00:00
Data variables: (12/23)
    uo          (time, lev, lat, lon) float32 dask.array<chunksize=(1, 1, 180, 360), meta=np.ndarray>
    vo          (time, lev, lat, lon) float32 dask.array<chunksize=(1, 1, 180, 360), meta=np.ndarray>
    so          (time, lev, lat, lon) float32 dask.array<chunksize=(1, 1, 180, 360), meta=np.ndarray>
    temp        (time, lev, lat, lon) float32 dask.array<chunksize=(1, 1, 180, 360), meta=np.ndarray>
    ho          (time, lev1, lat, lon) float32 dask.array<chunksize=(1, 9, 180, 360), meta=np.ndarray>
    pbo         (time, lat, lon) float32 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
    ...          ...
    LwLatSens   (time, lat, lon) float32 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
    Heat_PmE    (time, lat, lon) float32 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
    average_T1  (time) datetime64[ns] dask.array<chunksize=(1,), meta=np.ndarray>
    average_T2  (time) datetime64[ns] dask.array<chunksize=(1,), meta=np.ndarray>
    average_DT  (time) timedelta64[ns] dask.array<chunksize=(1,), meta=np.ndarray>
    time_bnds   (time, nv) object dask.array<chunksize=(1, 2), meta=np.ndarray>
Attributes:
    NumFilesInSet:  1
    title:          UFS
    grid_type:      regular
    grid_tile:      N/A

## Storing the dataset

The dataset has more variables than we wanted, but these are trimmed when we call [.store_dataset()](generated/ufs2arco.MOM6Dataset.store_dataset.rst).

Any arguments beyond the dataset are passed to `xarray.to_zarr`.

By default, dask (which we're accessing through xarray under the hood) stores data to zarr using the chunksizes that are being used currently by dask (i.e., how the data are viewed in memory, currently).
However, we will change this layout based on the `chunks_out` option in our configuration yaml.

In [14]:
reader.chunks_out

{'time': 1, 'z_l': -1, 'z_i': -1, 'yh': 60, 'xh': 60, 'yq': 60, 'xq': 60}

In [15]:
# overwrite chunks for regridded dataset
reader.chunks_out = {"time": 1, "lev": -1, "lev1": -1, "lon": 60, "lat": 60}

In [16]:
reader.store_dataset(ds, mode="w")

Stored dataset at example-replay-1deg/mom6.zarr


In [17]:
xds = xr.open_zarr(reader.data_path)

In [18]:
xds

<xarray.Dataset>
Dimensions:  (time: 2, lon: 360, lat: 180, lev: 75)
Coordinates:
    cftime   (time) object dask.array<chunksize=(2,), meta=np.ndarray>
    ftime    (time) timedelta64[ns] dask.array<chunksize=(2,), meta=np.ndarray>
  * lat      (lat) float64 -89.24 -88.25 -87.25 -86.26 ... 87.25 88.25 89.24
  * lev      (lev) float64 0.5154 1.571 2.687 ... 5.495e+03 5.698e+03 5.902e+03
  * lon      (lon) float64 0.0 1.0 2.0 3.0 4.0 ... 355.0 356.0 357.0 358.0 359.0
  * time     (time) datetime64[ns] 1994-01-01 1994-01-01T06:00:00
Data variables:
    SSH      (time, lon, lat) float32 dask.array<chunksize=(1, 60, 60), meta=np.ndarray>
    so       (time, lev, lon, lat) float32 dask.array<chunksize=(1, 75, 60, 60), meta=np.ndarray>
    temp     (time, lev, lon, lat) float32 dask.array<chunksize=(1, 75, 60, 60), meta=np.ndarray>
    uo       (time, lev, lon, lat) float32 dask.array<chunksize=(1, 75, 60, 60), meta=np.ndarray>
    vo       (time, lev, lon, lat) float32 dask.array<chunksize=(1, 75, 60, 60), meta=np.ndarray>
Attributes:
    NumFilesInSet:  1
    grid_tile:      N/A
    grid_type:      regular
    title:          UFS

We can see that this only has the data variables we asked for in the config yaml file.

In [19]:
xds.temp

<xarray.DataArray 'temp' (time: 2, lev: 75, lon: 360, lat: 180)>
dask.array<open_dataset-temp, shape=(2, 75, 360, 180), dtype=float32, chunksize=(1, 75, 60, 60), chunktype=numpy.ndarray>
Coordinates:
    cftime   (time) object dask.array<chunksize=(2,), meta=np.ndarray>
    ftime    (time) timedelta64[ns] dask.array<chunksize=(2,), meta=np.ndarray>
  * lat      (lat) float64 -89.24 -88.25 -87.25 -86.26 ... 87.25 88.25 89.24
  * lev      (lev) float64 0.5154 1.571 2.687 ... 5.495e+03 5.698e+03 5.902e+03
  * lon      (lon) float64 0.0 1.0 2.0 3.0 4.0 ... 355.0 356.0 357.0 358.0 359.0
  * time     (time) datetime64[ns] 1994-01-01 1994-01-01T06:00:00
Attributes:
    long_name:      Potential Temperature
    regrid_method:  bilinear
    units:          degC

And we can see this has a different chunking scheme than the original dataset, it has what we asked for in the config yaml.

In [20]:
reader.chunks_out

{'time': 1, 'lev': -1, 'lev1': -1, 'lon': 60, 'lat': 60}

<div class="alert alert-info">

Note:

This chunking scheme is not necessarily desirable, as the chunk sizes are quite small. This is simply used as an example to illustrate that we have control
over the chunk sizes.

</div>

## Cleanup

In [21]:
from shutil import rmtree

In [22]:
rmtree(reader.data_path)